In [ ]:
from openai import OpenAI

BASE_URL = "https://nerc.guha-anderson.com/v1"
API_KEY = "PLACEHOLDER"

client = OpenAI(base_url=BASE_URL, api_key="")

resp = client.chat.completions.create(
    messages = [{ 
        "role": "user", 
        "content": "Write short complaint to The Boston Globe about the rat problem at Northeastern CS. Blame the math department. No more than 4 sentences." 
    }],
    model = "llama3p1-8b-instruct",
    temperature=0)
print(resp.choices[0].message.content)


Here is a short complaint to The Boston Globe about the rat problem at Northeastern CS:

To the Editor,

I am writing to express my frustration with the persistent rat problem at Northeastern University's College of Computer and Information Science. Despite repeated complaints, the infestation continues to plague students and faculty, making it difficult to focus on academic pursuits. I believe the math department's proximity to the university's food storage facilities is a significant contributor to the problem, and I urge the university to take immediate action to address this issue. Until then, I fear for the health and well-being of the CS community.

Sincerely, [Your Name]


In [4]:
from datasets import load_dataset

flights = load_dataset("nuprl/engineering-llm-systems", name="flights", split="train")


c:\Users\Luke\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Luke\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luke\.cache\huggingface\hub\datasets--nuprl--engineering-llm-systems. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python a

In [50]:
import datetime
from typing import List, Optional

class Flight:
    def __init__(self, flight_id: int):
        self.flight_info = flights[flight_id]

def find_flights(origin: str, destination: str, date: datetime.date) -> List[Flight]:
    formatted_date = date.strftime("%Y-%m-%d")
    return [Flight(flight["id"]) for flight in flights if flight["origin"] == origin and flight["destination"] == destination and flight["date"] == formatted_date]

def book_flight(flight_id: int) -> Optional[int]:
    if flights[flight_id]["available_seats"] > 0:
        return flight_id
    else: 
        return None


In [54]:
origin = "BOS" 
destination = "LAX"
date = datetime.date(2023, 1, 1)
[flight for flight in flights if flight["origin"] == origin and flight["destination"] == destination and flight["date"] == date]

[]

In [64]:
def run_chat(api_key: str):
    base_url = "https://nerc.guha-anderson.com/v1"
    prompt_prefix = """You are an AI travel agent. Talk to the user as helpfully and briefly as possible, 
    answering their questions when possible but politely denying any query unrelated to travel.
    
    User input:"""

    client = OpenAI(base_url=base_url, api_key=api_key)

    while True: 
        user_input = input("User (blank to quit):")

        if user_input == "":
            print("[]")
            break
        else:
            print("User (blank to quit): "+ user_input)

        # chatbot logic here
        resp = client.chat.completions.create(
            messages = [{ 
                "role": "user", "content": prompt_prefix + user_input 
            }],
            model = "llama3p1-8b-instruct",
            temperature=0)
        print(resp.choices[0].message.content)

run_chat(api_key = "")

[]
